# Sarcasm Detection using Pretrained Language Models

This notebook implements sarcasm detection on Reddit political comments using transformer-based pretrained language models.

In [ ]:
# Mount Google Drive and set up paths
from google.colab import drive
import os
import sys

drive.mount("/content/drive")
base_dir = "/content/drive/MyDrive/SarcasmDetection"
sys.path.append(base_dir)
os.chdir(base_dir)

# Data paths
train_data_balanced_path = os.path.join(base_dir, "train-balanced-sarcasm.csv")
test_data_balanced_path = os.path.join(base_dir, "test-balanced.csv")
test_data_unbalanced_path = os.path.join(base_dir, "test-unbalanced.csv")

In [ ]:
# Install required libraries
!pip install transformers datasets evaluate scikit-learn pandas numpy matplotlib seaborn torch

## 1. Data Loading and Exploration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import warnings
warnings.filterwarnings('ignore')

# Load the datasets
train_data = pd.read_csv(train_data_balanced_path)
test_data_balanced = pd.read_csv(test_data_balanced_path)
test_data_unbalanced = pd.read_csv(test_data_unbalanced_path)

# Display dataset information
print(f"Training data shape: {train_data.shape}")
print(f"Test data (balanced) shape: {test_data_balanced.shape}")
print(f"Test data (unbalanced) shape: {test_data_unbalanced.shape}")

# Display sample data
train_data.head()

In [ ]:
# Check for missing values
print("\nMissing values in training data:")
print(train_data.isnull().sum())

# Class distribution
plt.figure(figsize=(8, 5))
sns.countplot(x='label', data=train_data)
plt.title('Distribution of Sarcastic vs Non-Sarcastic Comments')
plt.xlabel('Sarcasm (1=Yes, 0=No)')
plt.ylabel('Count')
plt.show()

# Check comment length distribution
train_data['comment_length'] = train_data['comment'].fillna('').apply(len)
plt.figure(figsize=(10, 6))
sns.histplot(data=train_data, x='comment_length', hue='label', bins=50, kde=True)
plt.title('Comment Length Distribution')
plt.xlabel('Comment Length (characters)')
plt.ylabel('Count')
plt.xlim(0, 1000)  # Limiting x-axis for better visualization
plt.show()

## 2. Data Preprocessing

In [ ]:
# Fill missing comments with empty string
train_data['comment'] = train_data['comment'].fillna('')
test_data_balanced['comment'] = test_data_balanced['comment'].fillna('')

# Basic preprocessing - remove deleted comments
train_data = train_data[~train_data['comment'].str.contains('[deleted]|[removed]', case=False, regex=True)]
test_data_balanced = test_data_balanced[~test_data_balanced['comment'].str.contains('[deleted]|[removed]', case=False, regex=True)]

# Check distribution after preprocessing
print(f"Training data shape after preprocessing: {train_data.shape}")
print(f"Test data shape after preprocessing: {test_data_balanced.shape}")

## 3. Context-Aware Preprocessing

We enhance the model's understanding by including parent comments as context when available.

In [ ]:
# Add context from parent comments where available
def prepare_context_aware_input(row):
    if pd.notna(row['parent_comment']) and row['parent_comment'] != '':
        # Truncate parent comment if too long
        parent = row['parent_comment'][:500] + '...' if len(row['parent_comment']) > 500 else row['parent_comment']
        return f"Parent: {parent} Comment: {row['comment']}"
    else:
        return f"Comment: {row['comment']}"

# Apply to create context-aware inputs
train_data['context_input'] = train_data.apply(prepare_context_aware_input, axis=1)
test_data_balanced['context_input'] = test_data_balanced.apply(prepare_context_aware_input, axis=1)

# Display a few examples
print("Context-aware inputs:")
for i in range(3):
    print(f"Example {i+1}: {train_data['context_input'].iloc[i][:200]}...")
    print(f"Label: {'Sarcastic' if train_data['label'].iloc[i] == 1 else 'Not sarcastic'}")
    print("-"*80)

## 4. Transformer Model Implementation

We'll use the Hugging Face Transformers library to fine-tune a pretrained model for sarcasm detection.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import evaluate

# Define model name - you can change this to other models
model_name = "roberta-base"  # Options: "distilbert-base-uncased", "albert-base-v2"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2
)

# Print model info
print(f"Using model: {model_name}")
print(f"Model size: {sum(p.numel() for p in model.parameters())/1e6:.2f}M parameters")

In [ ]:
# Prepare dataset
# Use a smaller subset for faster training in this notebook
sample_size = 100000  # Adjust based on your computational resources
if len(train_data) > sample_size:
    train_data_sample = train_data.sample(sample_size, random_state=42)
    print(f"Using {sample_size} samples from training data")
else:
    train_data_sample = train_data
    print(f"Using all {len(train_data)} training samples")

# Split into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data_sample['context_input'].tolist(),
    train_data_sample['label'].tolist(),
    test_size=0.1,
    stratify=train_data_sample['label'],
    random_state=42
)

In [ ]:
# Tokenize data
# We use a max length of 256 tokens - adjust based on your dataset
max_length = 256

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(test_data_balanced['context_input'].tolist(), 
                         truncation=True, padding=True, max_length=max_length)

# Create torch datasets
class SarcasmDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SarcasmDataset(train_encodings, train_labels)
val_dataset = SarcasmDataset(val_encodings, val_labels)
test_dataset = SarcasmDataset(test_encodings, test_data_balanced['label'].tolist())

In [ ]:
# Calculate class weights to handle imbalanced data
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print(f"Class weights: {class_weights_dict}")

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,  # Adjust as needed
    per_device_train_batch_size=16,  # Adjust based on your GPU memory
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    label_smoothing_factor=0.1,  # Helps with overconfidence
)

# Define metrics for evaluation
metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='weighted')

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# Train model
trainer.train()

In [ ]:
# Evaluate on validation set
val_results = trainer.evaluate()
print(f"Validation results: {val_results}")

# Evaluate on test set
test_results = trainer.evaluate(test_dataset)
print(f"Test results: {test_results}")

## 5. Detailed Model Evaluation

In [ ]:
# Get detailed predictions on test set
test_pred_output = trainer.predict(test_dataset)
test_preds = np.argmax(test_pred_output.predictions, axis=1)

# Create classification report
print("Classification Report:")
print(classification_report(test_data_balanced['label'], test_preds))

# Confusion Matrix
cm = confusion_matrix(test_data_balanced['label'], test_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix - Transformer Model')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

## 6. Error Analysis

Let's look at some examples the model got wrong.

In [ ]:
# Create DataFrame with predictions
test_df = test_data_balanced.copy()
test_df['predicted'] = test_preds
test_df['correct'] = test_df['label'] == test_df['predicted']

# False positives (predicted sarcastic but not)
false_positives = test_df[(test_df['predicted'] == 1) & (test_df['label'] == 0)]
print(f"Number of false positives: {len(false_positives)}")
print("\nExamples of false positives (predicted sarcastic but not):")
for i in range(min(5, len(false_positives))):
    print(f"Example {i+1}: {false_positives['comment'].iloc[i][:200]}...")
    print("-"*80)

# False negatives (predicted not sarcastic but is)
false_negatives = test_df[(test_df['predicted'] == 0) & (test_df['label'] == 1)]
print(f"\nNumber of false negatives: {len(false_negatives)}")
print("\nExamples of false negatives (predicted not sarcastic but is):")
for i in range(min(5, len(false_negatives))):
    print(f"Example {i+1}: {false_negatives['comment'].iloc[i][:200]}...")
    print("-"*80)

## 7. Interpreting Model Predictions

Here we'll analyze a few examples to see what the model focuses on.

In [ ]:
from transformers import pipeline

# Create a classifier from our fine-tuned model
classifier = pipeline(
    "text-classification", 
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

# Function to perform word importance analysis
def analyze_word_importance(text, classifier, top_n=10):
    # Get base prediction
    base_pred = classifier(text)[0]
    # Split into words
    words = text.split()
    word_importance = []
    
    for i in range(len(words)):
        # Skip if the word is too short
        if len(words[i]) <= 2:
            continue
            
        # Create a version with this word masked
        masked_words = words.copy()
        masked_words[i] = '[MASK]'
        masked_text = ' '.join(masked_words)
        
        # Check prediction change
        masked_result = classifier(masked_text)[0]
        importance = abs(base_pred['score'] - masked_result['score'])
        word_importance.append((words[i], importance))
    
    # Sort by importance
    word_importance.sort(key=lambda x: x[1], reverse=True)
    return word_importance[:top_n]

# Choose some examples to analyze
examples = [
    test_df[test_df['label'] == 1]['context_input'].iloc[0],  # Sarcastic 
    test_df[test_df['label'] == 0]['context_input'].iloc[0],  # Non-sarcastic
    false_positives['context_input'].iloc[0],  # False positive
    false_negatives['context_input'].iloc[0]   # False negative
]

# Analyze each example
for i, example in enumerate(examples):
    print(f"Example {i+1}: {example[:200]}...")
    result = classifier(example)[0]
    print(f"Prediction: {result['label']}, Confidence: {result['score']:.4f}")
    
    important_words = analyze_word_importance(example, classifier)
    print("\nMost important words:")
    for word, score in important_words:
        print(f"{word}: {score:.4f}")
    print("-"*80)

## 8. Subreddit Analysis with Transformer Results

In [ ]:
# Add predictions to train dataset (using a subsample for efficiency)
# This requires some modifications since we might not have predictions for all data
sample_train_texts = train_data.sample(5000, random_state=42)['context_input'].tolist()
sample_train_encodings = tokenizer(sample_train_texts, truncation=True, padding=True, max_length=max_length)
sample_train_dataset = SarcasmDataset(
    sample_train_encodings, 
    [0] * len(sample_train_texts)  # Dummy labels, we only need predictions
)

# Get predictions
sample_pred_output = trainer.predict(sample_train_dataset)
sample_probs = sample_pred_output.predictions
sample_preds = np.argmax(sample_probs, axis=1)

# Get the confidence scores for the positive class (sarcasm)
sample_sarcasm_probs = sample_probs[:, 1]

# Create a DataFrame with indices
train_sample_idx = train_data.sample(5000, random_state=42).index
pred_df = pd.DataFrame({
    'index': train_sample_idx,
    'predicted': sample_preds,
    'sarcasm_confidence': sample_sarcasm_probs
})

# Merge with original data
train_with_preds = train_data.loc[pred_df['index']].copy()
train_with_preds['predicted'] = pred_df['predicted'].values
train_with_preds['sarcasm_confidence'] = pred_df['sarcasm_confidence'].values

# Analyze by subreddit
subreddit_analysis = train_with_preds.groupby('subreddit').agg({
    'label': 'mean',  # True sarcasm rate
    'predicted': 'mean',  # Predicted sarcasm rate
    'sarcasm_confidence': 'mean',  # Average confidence
    'index': 'count'  # Count of comments
}).rename(columns={'index': 'count', 'label': 'true_sarcasm_rate', 'predicted': 'predicted_sarcasm_rate'})

# Filter for subreddits with enough comments
min_comments = 20
subreddit_analysis = subreddit_analysis[subreddit_analysis['count'] >= min_comments]

# Sort by true sarcasm rate
subreddit_analysis = subreddit_analysis.sort_values('true_sarcasm_rate', ascending=False)

# Display top 20 subreddits
print("Top 20 subreddits by true sarcasm rate:")
subreddit_analysis.head(20)

In [ ]:
# Plot subreddit analysis
top_n = 15
plt.figure(figsize=(12, 8))

# Get top N subreddits by sarcasm rate with at least min_comments
top_subreddits = subreddit_analysis.head(top_n)

# Plot true vs predicted sarcasm rates
top_subreddits[['true_sarcasm_rate', 'predicted_sarcasm_rate']].plot(
    kind='bar', 
    color=['firebrick', 'steelblue']
)

plt.title(f'Top {top_n} Subreddits by Sarcasm Rate (min {min_comments} comments)')
plt.ylabel('Sarcasm Rate')
plt.xlabel('Subreddit')
plt.xticks(rotation=45, ha='right')
plt.legend(['True Sarcasm Rate', 'Predicted Sarcasm Rate'])
plt.tight_layout()
plt.show()

## 9. Creating a Practical Sarcasm Detection Function

In [ ]:
def predict_sarcasm(text, context=None, classifier=None):
    """Predict whether text is sarcastic"""
    if classifier is None:
        # Load model if not provided - this assumes the model is already trained
        classifier = pipeline(
            "text-classification", 
            model=model,
            tokenizer=tokenizer
        )
    
    # Prepare input with context if available
    if context:
        input_text = f"Parent: {context} Comment: {text}"
    else:
        input_text = f"Comment: {text}"
    
    # Get prediction
    result = classifier(input_text)[0]
    is_sarcastic = result['label'] == 'LABEL_1'
    
    # Return formatted result
    return {
        "text": text,
        "is_sarcastic": is_sarcastic,
        "confidence": result['score'],
        "context": context
    }

# Example usage
examples = [
    {"text": "Yeah, sure, that's definitely going to work out great.", 
     "context": "We should cut taxes for the rich to stimulate the economy."},
    {"text": "This new policy will significantly reduce our carbon footprint.", 
     "context": "The government announced new climate regulations today."}
]

for example in examples:
    result = predict_sarcasm(example["text"], example["context"], classifier)
    print(f"Text: {result['text']}")
    print(f"Is sarcastic: {result['is_sarcastic']} (Confidence: {result['confidence']:.4f})")
    print(f"Context: {result['context']}\n")

## 10. Comparing with Classical ML Models

If you have results from the previous notebook with classical models, you can compare them here.

In [ ]:
# Define your baseline model results here
# This is a placeholder - fill in with your actual results from the previous notebook
baseline_results = {
    'Bag of Words': {'accuracy': 0.70, 'f1': 0.69},
    'TF-IDF': {'accuracy': 0.72, 'f1': 0.71},
    'Combined': {'accuracy': 0.74, 'f1': 0.73}
}

# Add transformer results
transformer_accuracy = test_results['eval_accuracy']
transformer_f1 = test_results['eval_f1']
baseline_results['Transformer'] = {'accuracy': transformer_accuracy, 'f1': transformer_f1}

# Plot comparison
models = list(baseline_results.keys())
accuracies = [baseline_results[model]['accuracy'] for model in models]
f1_scores = [baseline_results[model]['f1'] for model in models]

plt.figure(figsize=(12, 6))
x = np.arange(len(models))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(x - width/2, accuracies, width, label='Accuracy', color='darkblue')
ax.bar(x + width/2, f1_scores, width, label='F1 Score', color='darkred')

# Add value labels
for i, v in enumerate(accuracies):
    ax.text(i - width/2, v + 0.01, f'{v:.3f}', ha='center')
    
for i, v in enumerate(f1_scores):
    ax.text(i + width/2, v + 0.01, f'{v:.3f}', ha='center')

ax.set_ylabel('Score')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()

plt.tight_layout()
plt.show()

## 11. Save the Model for Future Use

In [ ]:
# Save model and tokenizer
save_path = os.path.join(base_dir, "transformer_sarcasm_model")
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model and tokenizer saved to {save_path}")

## 12. Conclusion

In this notebook, we've demonstrated how to use a transformer-based pretrained language model to detect sarcasm in political comments. The key advantages of this approach include:

1. **Context awareness**: By incorporating parent comments as context, the model can better understand subtle sarcasm cues.
2. **Transfer learning**: Using pretrained models provides strong language understanding capabilities out of the box.
3. **Performance**: Transformer models typically outperform classical ML approaches for this type of task.

Practical applications include:
- Social media moderation to identify potentially misinterpreted content
- Political sentiment analysis that can distinguish between genuine and sarcastic opinions
- Research tools to analyze communication patterns in political discourse